In [6]:
import pandas, numpy, math, pickle, urllib, json

def get_driving_distance_osm(str_lat_comma_long1,str_lat_comma_long2):
    query_args = { 'fromPlace':str_lat_comma_long1, 'toPlace':str_lat_comma_long2,'mode':'CAR' }
    encoded_args = urllib.urlencode(query_args)
    #print 'Encoded:', encoded_args
    url = 'http://13.218.151.99:8080/otp/routers/default/plan?' + encoded_args
    temp = json.load(urllib.urlopen(url))
    return temp['plan']['itineraries'][0]['legs'][0]['distance']

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km

#Distances in Simulated data
import numpy, pickle
from profit_maximization import euclidean
flag_sim_dist_stat = False
if flag_sim_dist_stat:
    data = pickle.load(open('../../../Xharecost_MS_annex/plot_data.pkl','rb'))
    instance = data['instance_base']
    all_requests = instance['all_requests']
    distances = numpy.zeros(len(all_requests))
    for i in all_requests:
        distances[i] = euclidean(all_requests[i]['orig'],all_requests[i]['dest'])
    distDF = pandas.DataFrame(data=distances,index=range(len(all_requests)),columns=['dist'])
    #print distDF.describe()
    print distDF.median()
# dist    49.527744
# dtype: float64

             dist
count  100.000000
mean    52.211287
std     25.948891
min      3.162278
25%     31.988244
50%     49.527744
75%     69.020105
max    112.507778
dist    49.527744
dtype: float64


In [7]:
df0 = pandas.read_csv('../../../Xharecost_MS_annex/hour9.csv',usecols= ['pickup_datetime','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'])

df = df0[(abs(df0.pickup_latitude-0) > 1e-5) & \
         (abs(df0.pickup_longitude-0) > 1e-5) & \
         (abs(df0.dropoff_latitude-0) > 1e-5) & \
         (abs(df0.dropoff_longitude-0) > 1e-5)]
#print df.describe()

In [45]:
flag_debug = True
if flag_debug:
    str_lat_comma_long1 = '40.616276,-74.017921'
    str_lat_comma_long2 = '40.865898,-73.697060'
    #print get_driving_distance_osm(str_lat_comma_long1,str_lat_comma_long2)
    print haversine(-74.017921,40.616276,-73.697060,40.865898)

38.7205449962


In [46]:
38.7*1.6

61.92000000000001

In [17]:
df['pickup_datetime'] = pandas.to_datetime(df['pickup_datetime'])
for i in range(60):
    df1 = df[(df['pickup_datetime'].dt.minute >= i) & (df['pickup_datetime'].dt.minute < i+1)]
    #print 'minute: {0} count: {1}'.format(i,len(df1))

C:\Users\q4fj4lj9\AppData\Local\Continuum\Miniconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [41]:
float(df1.iloc[[200]]['dropoff_latitude'])

40.733109000000006